<h2> Featurizing text data with tfidf weighted word-vectors </h2>

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import time
import warnings
import numpy as np
from nltk.corpus import stopwords
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
warnings.filterwarnings("ignore")
import sys
import os 
import pandas as pd
import numpy as np
from tqdm import tqdm

import spacy

In [3]:
df = pd.read_csv("train.csv")

In [4]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
# merge texts
questions = list(df['question1']) + list(df['question2'])

tfidf = TfidfVectorizer(lowercase=False, )
tfidf.fit_transform(questions)

# dict key:word and value:tf-idf score
word2tfidf = dict(zip(tfidf.get_feature_names(), tfidf.idf_))
tfidf_words = set(tfidf.get_feature_names())
dictionary = dict(zip(tfidf.get_feature_names(), list(tfidf.idf_)))

- After we find TF-IDF scores, we convert each question to a weighted average of word2vec vectors by these scores.
- here we use a pre-trained GLOVE model 

In [6]:
df

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
...,...,...,...,...,...,...
404285,404285,433578,379845,How many keywords are there in the Racket prog...,How many keywords are there in PERL Programmin...,0
404286,404286,18840,155606,Do you believe there is life after death?,Is it true that there is life after death?,1
404287,404287,537928,537929,What is one coin?,What's this coin?,0
404288,404288,537930,537931,What is the approx annual cost of living while...,I am having little hairfall problem but I want...,0


In [11]:
import pickle
#please use below code to load glove vectors 
with open('glove_vectors', 'rb') as f:
    model = pickle.load(f)
    glove_words =  set(model.keys())

In [14]:
tfidf_w2v_question_1= []; # the avg-w2v for each sentence/review is stored in this list
for sentence in tqdm(df['question1']): # for each review/sentence
    vector = np.zeros(300) # as word vectors are of zero length
    tf_idf_weight =0; # num of words with a valid vector in the sentence/review
    for word in sentence.split(): # for each word in a review/sentence
        if (word in glove_words) and (word in tfidf_words):
            vec = model[word] # getting the vector for each word
            # here we are multiplying idf value(dictionary[word]) and the tf value((sentence.count(word)/len(sentence.split())))
            tf_idf = dictionary[word]*(sentence.count(word)/len(sentence.split())) # getting the tfidf value for each word
            vector += (vec * tf_idf) # calculating tfidf weighted w2v
            tf_idf_weight += tf_idf
    if tf_idf_weight != 0:
        vector /= tf_idf_weight
    tfidf_w2v_question_1.append(vector)

print(len(tfidf_w2v_question_1))
print(len(tfidf_w2v_question_1[0]))

100%|████████████████████████████████████████████████████████████████████████| 404290/404290 [01:04<00:00, 6276.01it/s]

404290
300


In [16]:
tfidf_w2v_question_2= []; # the avg-w2v for each sentence/review is stored in this list
for sentence in tqdm(df['question2']): # for each review/sentence
    vector = np.zeros(300) # as word vectors are of zero length
    tf_idf_weight =0; # num of words with a valid vector in the sentence/review
    for word in sentence.split(): # for each word in a review/sentence
        if (word in glove_words) and (word in tfidf_words):
            vec = model[word] # getting the vector for each word
            # here we are multiplying idf value(dictionary[word]) and the tf value((sentence.count(word)/len(sentence.split())))
            tf_idf = dictionary[word]*(sentence.count(word)/len(sentence.split())) # getting the tfidf value for each word
            vector += (vec * tf_idf) # calculating tfidf weighted w2v
            tf_idf_weight += tf_idf
    if tf_idf_weight != 0:
        vector /= tf_idf_weight
    tfidf_w2v_question_2.append(vector)

print(len(tfidf_w2v_question_2))
print(len(tfidf_w2v_question_2[0]))

100%|████████████████████████████████████████████████████████████████████████| 404290/404290 [01:07<00:00, 5968.14it/s]

404290
300


In [20]:
df['tfidf_w2v_question_2'] = tfidf_w2v_question_2
df['tfidf_w2v_question_1'] = tfidf_w2v_question_1

In [24]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate,tfidf_w2v_question_2,tfidf_w2v_question_1
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,"[-0.07784363447344539, 0.18513011227426512, -0...","[-0.05428421575064088, 0.12906880972707951, -0..."
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,"[-0.012144351675695508, 0.043515074380045614, ...","[-0.10091133625160288, 0.17586209267592476, -0..."
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,"[-0.2568414313540464, -0.025848513916250822, 0...","[-0.20386655856523256, -0.14370405928535118, -..."
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,"[0.02140114059537408, 0.08944129046166255, -0....","[-0.27353072303289044, 0.1170396179381546, -0...."
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,"[-0.08250975993190177, 0.019007367216438812, 0...","[-0.14526175904621025, -0.09697306628316683, 0..."


In [21]:
df_nlp = pd.read_csv("nlp_features_train.csv")
df_fe = pd.read_csv("df_fe_without_preprocessing_train.csv")

In [26]:
df1 = df_nlp.drop(['qid1','qid2','question1','question2'],axis=1)
df2 = df_fe.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)

In [27]:
df3 = df.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)

In [28]:
df3_q1 = pd.DataFrame(df3.tfidf_w2v_question_1.values.tolist(), index= df3.index)

In [30]:
df3_q2 = pd.DataFrame(df3.tfidf_w2v_question_2.values.tolist(), index= df3.index)

In [31]:
# dataframe of nlp features
df1.head()

,id,is_duplicate,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,token_set_ratio,token_sort_ratio,fuzz_ratio,fuzz_partial_ratio
0,0,0,0.999980,0.833319,0.999983,0.999983,0.916659,0.785709,0.0,1.0,2.0,13.0,100,93,93,100
1,1,0,0.799984,0.399996,0.749981,0.599988,0.699993,0.466664,0.0,1.0,5.0,12.5,86,63,66,75
2,2,0,0.399992,0.333328,0.399992,0.249997,0.399996,0.285712,0.0,1.0,4.0,12.0,63,63,43,47
3,3,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,2.0,12.0,28,24,9,14
4,4,0,0.399992,0.199998,0.999950,0.666644,0.571420,0.307690,0.0,1.0,6.0,10.0,67,47,35,56


In [32]:
# data before preprocessing 
df2.head()

,id,freq_qid1,freq_qid2,q1len,q2len,q1_n_words,q2_n_words,word_Common,word_Total,word_share,freq_q1+q2,freq_q1-q2
0,0,1,1,66,57,14,12,10.0,23.0,0.434783,2,0
1,1,4,1,51,88,8,13,4.0,20.0,0.200000,5,3
2,2,1,1,73,59,14,10,4.0,24.0,0.166667,2,0
3,3,1,1,50,65,11,9,0.0,19.0,0.000000,2,0
4,4,3,1,76,39,13,7,2.0,20.0,0.100000,4,2


In [33]:
# Questions 1 tfidf weighted word2vec
df3_q1.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.054284,0.129069,-0.019582,-0.093599,0.059739,-0.128221,-3.313700,0.613361,-0.048382,-0.037377,...,-0.089340,-0.253683,0.179144,0.096986,0.030545,0.003065,0.064364,-0.102067,0.282534,-0.047377
1,-0.100911,0.175862,-0.008297,0.012398,-0.111981,0.050525,-3.372036,0.415174,-0.053273,-0.543833,...,-0.036245,-0.010889,0.225521,0.212017,0.086895,0.239466,0.120711,0.006904,0.195965,0.079456
2,-0.203867,-0.143704,-0.136954,-0.227616,0.096596,0.030507,-3.697412,0.687129,0.063790,-0.134584,...,0.343854,-0.157845,-0.094655,-0.120453,-0.094282,0.038055,-0.193141,-0.313417,0.059885,-0.204855
3,-0.273531,0.117040,-0.129420,-0.063649,-0.161661,-0.100724,-3.517240,-0.024248,0.164644,-0.245647,...,0.199109,-0.358276,-0.329825,-0.017442,0.033861,-0.078383,0.024264,0.286761,0.196753,-0.262993
4,-0.145262,-0.096973,0.108462,-0.340343,-0.126769,-0.010428,-2.024518,0.763158,0.034523,0.117497,...,-0.099987,0.172572,0.327414,0.214909,0.077680,-0.296927,-0.218308,0.034184,-0.143549,0.020201


In [34]:
# Questions 2 tfidf weighted word2vec
df3_q2.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.077844,0.185130,-0.125995,-0.018330,0.070562,-0.097759,-3.219032,0.651196,-0.079764,-0.017207,...,-0.108470,-0.172584,0.115508,0.165498,0.050041,0.104500,0.047697,-0.031508,0.253518,-0.018215
1,-0.012144,0.043515,-0.058075,0.055848,-0.049310,0.051657,-2.898511,0.108433,0.026987,-0.386361,...,-0.142902,-0.149380,-0.082208,-0.105020,-0.225894,0.029982,0.056607,-0.160301,0.197302,-0.089608
2,-0.256841,-0.025849,0.066133,-0.260259,0.033963,-0.005008,-3.407979,0.550632,0.128148,-0.064791,...,0.143160,-0.306466,-0.082052,-0.255014,-0.160884,0.154167,0.187339,-0.174920,-0.027716,-0.090748
3,0.021401,0.089441,-0.184650,-0.220473,-0.076607,0.019328,-2.836467,0.656580,0.060847,0.103786,...,-0.030402,-0.169257,-0.042085,-0.205133,0.041855,0.305579,0.271215,0.139906,0.173737,0.041718
4,-0.082510,0.019007,0.096250,-0.270339,-0.046538,-0.085152,-2.753134,0.071840,-0.008540,-0.335259,...,-0.250071,-0.378713,0.207525,0.161997,0.047324,-0.273545,-0.028424,0.038811,-0.071177,0.260552


In [35]:
print("Number of features in nlp dataframe :", df1.shape[1])
print("Number of features in preprocessed dataframe :", df2.shape[1])
print("Number of features in question1 w2v  dataframe :", df3_q1.shape[1])
print("Number of features in question2 w2v  dataframe :", df3_q2.shape[1])
print("Number of features in final dataframe  :", df1.shape[1]+df2.shape[1]+df3_q1.shape[1]+df3_q2.shape[1])

Number of features in nlp dataframe : 16
Number of features in preprocessed dataframe : 12
Number of features in question1 w2v  dataframe : 300
Number of features in question2 w2v  dataframe : 300
Number of features in final dataframe  : 628


In [36]:
df3_q1['id']=df1['id']
df3_q2['id']=df1['id']
df1  = df1.merge(df2, on='id',how='left')
df2  = df3_q1.merge(df3_q2, on='id',how='left')
result  = df1.merge(df2, on='id',how='left')
result.to_csv('final_features.csv')